In [1]:
"""
Simple example script fitting a model for synthetic data
"""
from __future__ import print_function
import jax.numpy as jnp
from jax import jit, random
import time
import matplotlib.pyplot as plt
import seaborn as sns

from codebase.vi.BBVI import DiagMvnBBVI
from codebase.gsmooth.opt import FilteredOptimization
from codebase.gsmooth.smoothers import AdamFilter
import CV as cvg
import models

%load_ext autoreload
%autoreload 2

/home/stevenxue/VI/src/codebase/models/frisk.py:28: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(os.path.join(data_dir, 'data/frisk/frisk_with_noise.dat'),
/home/stevenxue/VI/src/codebase/models/frisk.py:33: FutureWarning: The provided callable <built-in function sum> is currently using np.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string np.sum instead.
  popdf = df[['pop', 'precinct', 'eth']].groupby(['precinct', 'eth'])['pop'].apply(sum)


In [2]:
#########################################
# construct model function + vb object  #
#########################################
# lnpdf, D = make_model()
lnpdf, D, name = models.set_lnpdf("frisk")
key = random.PRNGKey(0)
th0 = random.normal(key, (D,))
print(lnpdf(th0))  # example use

# create bbvi object --- this just keeps references to lnpdf,
# grad(lnpdf), hvp(lnpdf), etc
vbobj = DiagMvnBBVI(lnpdf, D, lnpdf_is_vectorized=False)

# initialize params
key = random.PRNGKey(1)
lam0 = random.normal(key, (vbobj.num_variational_params,)) * 0.01 - 1
lam0 = lam0.at[D:].set(-3.)

TypeError: true_fun output and false_fun output must have identical types, got
DIFFERENT ShapedArray(float32[]) vs. ShapedArray(float32[1]).